<a href="https://colab.research.google.com/github/RamonFontelles/idealista/blob/main/idealista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [160]:
import requests
from bs4 import BeautifulSoup

In [161]:
from pandas import DataFrame
from logging import exception

# COMO ACCEDER A LA URL

In [162]:
# 2. Definir la URL y los Headers para simular un navegador
# La URL es la de la casa que quieres analizar.

#url = "https://www.idealista.com/inmueble/107170538/" #1
url = "https://www.idealista.com/inmueble/107170538/"

In [163]:
# ¡Esto es MUY importante! Los headers hacen que nuestra petición parezca
# que viene de un navegador real (Chrome en este caso) y no de un robot.
# Sin esto, Idealista nos bloqueará con un error 403 (Prohibido).
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "es-ES,es;q=0.9,en;q=0.8",
    "cache-control": "max-age=0",
    "cookie": "_pprv=eyJjb25zZW50Ijp7IjAiOnsibW9kZSI6Im9wdC1pbiJ9LCIxIjp7Im1vZGUiOiJvcHQtaW4ifSwiMiI6eyJtb2RlIjoib3B0LWluIn0sIjMiOnsibW9kZSI6Im9wdC1pbiJ9LCI0Ijp7Im1vZGUiOiJvcHQtaW4ifSwiNSI6eyJtb2RlIjoib3B0LWluIn0sIjYiOnsibW9kZSI6Im9wdC1pbiJ9LCI3Ijp7Im1vZGUiOiJvcHQtaW4ifX0sInB1cnBvc2VzIjpudWxsLCJfdCI6Im1sODdsanRmfG01anNvMmhmIn0%3D; _pcid=%7B%22browserId%22%3A%22m5jso2hdvbtfq55r%22%2C%22_t%22%3A%22ml87lk52%7Cm5jso2t2%22%7D; _pctx=%7Bu%7DN4IgrgzgpgThIC4B2YA2qA05owMoBcBDfSREQpAeyRCwgEt8oBJAE0RXSwH18yBbVAA4A7KgDWAVgDMAH36SAVhEoAmfNJABfIA; ABTasty=uid=y414qq4bt0a149dn&fst=1749216783138&pst=-1&cst=1749216783138&ns=1&pvt=1&pvis=1&th=; galleryHasBeenBoosted=true; askToSaveAlertPopUp=true; utag_main__sn=9; userUUID=bca4b99d-39c3-4391-b379-03fdc27b1869; SESSION=733bddb87ea43a90~9d08fc99-db59-4443-8b87-dd177c345553; contact9d08fc99-db59-4443-8b87-dd177c345553=\"{'maxNumberContactsAllow':10}\"; cookieSearch-1=\"/venta-viviendas/crevillente-alicante/:1750184635582\"; _last_search=officialZone; utag_main__prevEventLink=; __rtbh.uid=%7B%22eventType%22%3A%22uid%22%2C%22id%22%3A%22unknown%22%2C%22expiryDate%22%3A%222026-06-17T18%3A41%3A50.369Z%22%7D; __rtbh.lid=%7B%22eventType%22%3A%22lid%22%2C%22id%22%3A%22c0IUCy4I0E17TZnVtdQZ%22%2C%22expiryDate%22%3A%222026-06-17T18%3A41%3A50.370Z%22%7D; datadome=dnToYColvjIyEAcN5tDn_tStDkoVtG2tu4lTBWaT1im6t35B06PKn9gV4vL9Pk04JUDWzDHn6No3qHDcGvsmXHNPIZcwIvm1lKPlU6BRu~tL1EIzrM_PMkMgFt9jdtk5",
    "priority": "u=0, i",
    "referer": "https://www.idealista.com/venta-viviendas/crevillente-alicante/",
    "sec-ch-device-memory": "8",
    "sec-ch-ua": "\"Google Chrome\";v=\"137\", \"Chromium\";v=\"137\", \"Not/A)Brand\";v=\"24\"",
    "sec-ch-ua-arch": "\"\"",
    "sec-ch-ua-full-version-list": "\"Google Chrome\";v=\"137.0.7151.104\", \"Chromium\";v=\"137.0.7151.104\", \"Not/A)Brand\";v=\"24.0.0.0\"",
    "sec-ch-ua-mobile": "?1",
    "sec-ch-ua-model": "\"Nexus 5\"",
    "sec-ch-ua-platform": "\"Android\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Mobile Safari/537.36"
}

# 3. Hacer la petición a la página web
print("Accediendo a la página de Idealista...")
response = requests.get(url, headers=headers)

Accediendo a la página de Idealista...


# 3. Parsear el HTML con BeautifulSoup


In [164]:
soup = BeautifulSoup(response.content, 'lxml')

In [165]:
# Otra manera de sacar el titulo
titulo =  soup.find("span", {"class":"main-info__title-main"})
titulo.text.strip()

'Casa o chalet independiente en venta en calle San Antonio de la Florida'

In [166]:
# 4. Crear un diccionario vacío para almacenar los datos
datos_inmueble = {}

# 5. Extraer datos principales (Título, Precio, Descripción)
print("Extrayendo datos principales...")
# a) Extraer el Título
# Buscamos la etiqueta <h1> que tiene la clase 'main-info__title-main'
title_element = soup.find("h1").find("span").text
# Limpiamos el texto de espacios extra al principio o al final
#titulo = title_element.text.strip() if title_element else "Título no encontrado"
titulo = title_element
datos_inmueble['titulo'] = titulo
titulo



Extrayendo datos principales...


'Casa o chalet independiente en venta en calle San Antonio de la Florida'

In [167]:
# PRECIO
precio = soup.find("span", {"class":"info-data-price"}).text.strip()
datos_inmueble['precio'] = precio.replace(".", "").replace("€", "").strip()

datos_inmueble['precio']

'397000'

In [168]:
# UBICACIÓN
ubicacion = soup.find("div", {"id":'headerMap'}).text.strip().strip()

# Elimina la palabra "Ubicación" y convierte saltos de línea múltiples en uno solo
limpia = ubicacion.replace("Ubicación", "").replace("\n", ",")

# Elimina comas duplicadas y espacios redundantes
datos_inmueble['ubicacion'] = ','.join([x.strip() for x in limpia.split(",") if x.strip()])

datos_inmueble['ubicacion']

'Calle San Antonio de la Florida,Crevillente,Baix Vinalopó,Alicante'

In [169]:
# Para la descripción, buscamos el contenedor y luego el párrafo dentro
descripcion_container = soup.find('div', class_='adCommentsLanguage')
if descripcion_container:
    # Usamos .replace() para convertir los <br/> en saltos de línea legibles
    datos_inmueble['descripcion'] = descripcion_container.find('p').get_text(separator='\n').strip()
datos_inmueble['descripcion']

'¿Vivir en la cumbre y tener la ciudad a 5 minutos? ¡Es posible! \nDescubre esta exclusiva villa en La Florida, Crevillente, donde la tranquilidad de la naturaleza se combina con la comodidad de la ciudad. Con 575 m² construidos sobre una parcela de 2.580 m², esta vivienda unifamiliar de tres plantas ofrece vistas espectaculares al valle y al paraje natural del Hondo, lejos del ruido urbano. \nCaracterísticas destacadas: \nUbicación privilegiada: A solo 5 minutos de Crevillente, con acceso rápido a servicios, comercios y ocio. \nDiseño rústico y elegante: Integrada en la montaña, con tres plantas (dos sobre rasante y un sótano con acceso para vehículos). \nAmplias terrazas y zonas exteriores: 200 m² de terraza, piscina de 60 m², jacuzzi, barbacoa y huerta ecológica. \nInteriores espaciosos: Salón-comedor con chimenea y vistas de 180º, cocina con barra americana, bodega para 840 botellas, y cuatro habitaciones (incluida una suite principal). \nBuhardilla multifuncional: 61 m² con terraz

In [170]:
# CARACTERÍSTICAS PRINCIPALES
datos_inmueble['caracteristicas'] = {}
caracteristicas = soup.find("div", {"class":'info-features'}).text.strip().strip()
caracteristicas = caracteristicas.replace("\n", ",")
datos_inmueble['caracteristicas']['principales'] = ','.join([x.strip() for x in caracteristicas.split(",") if x.strip()])
datos_inmueble['caracteristicas']['principales']

'580 m²,4 hab.,Garaje incluido'

In [171]:

# 6. Extraer Características (Básicas, Equipamiento, Certificado)


# Buscamos el contenedor principal de todas las características
details_container = soup.find('div', class_='details-property')

if details_container:
    # a) Características básicas
    basic_features_div = details_container.find('div', class_='details-property-feature-one')
    if basic_features_div:
        items = basic_features_div.find_all('li')
        datos_inmueble['caracteristicas']['basicas'] = [item.text.strip() for item in items]

    # b) Equipamiento y Certificado energético
    extra_features_div = details_container.find('div', class_='details-property-feature-two')
    if extra_features_div:
        # Equipamiento
        h2_equipamiento = extra_features_div.find('h2', string='Equipamiento')
        if h2_equipamiento:
            ul_equipamiento = h2_equipamiento.find_next_sibling('div').find('ul')
            items_equip = ul_equipamiento.find_all('li')
            datos_inmueble['caracteristicas']['equipamiento'] = [item.text.strip() for item in items_equip]

        # Certificado energético
        h2_certificado = extra_features_div.find('h2', string='Certificado energético')
        if h2_certificado:
            datos_inmueble['caracteristicas']['certificado_energetico'] = {}
            ul_certificado = h2_certificado.find_next_sibling('div').find('ul')
            # Extraer Consumo
            try:
                consumo_li = ul_certificado.find('span', string='Consumo: ').find_parent('li')
                # El truco: la calificación está en la clase del icono
                clase_consumo = consumo_li.find('span', class_=lambda x: x and x.startswith('icon-energy-')).get('class')[-1]
                calificacion_consumo = clase_consumo.split('-')[-1].upper()
                datos_inmueble['caracteristicas']['certificado_energetico']['consumo'] = calificacion_consumo
            except:
                datos_inmueble['caracteristicas']['certificado_energetico']['consumo'] = "No indicado"
            # Extraer Emisiones
            try:
              emisiones_li = ul_certificado.find('span', string='Emisiones: ').find_parent('li')
              clase_emisiones = emisiones_li.find('span', class_=lambda x: x and x.startswith('icon-energy-')).get('class')[-1]
              calificacion_emisiones = clase_emisiones.split('-')[-1].upper()
              datos_inmueble['caracteristicas']['certificado_energetico']['emisiones'] = calificacion_emisiones
            except:
              datos_inmueble['caracteristicas']['certificado_energetico']['emisiones'] = "No indicado"

# --- FIN DE LA EXTRACCIÓN ---

# 7. Imprimir el diccionario resultante de forma legible
print("\n--- DICCIONARIO CON LOS DATOS EXTRAÍDOS ---")
print(datos_inmueble)


--- DICCIONARIO CON LOS DATOS EXTRAÍDOS ---
{'titulo': 'Casa o chalet independiente en venta en calle San Antonio de la Florida', 'precio': '397000', 'ubicacion': 'Calle San Antonio de la Florida,Crevillente,Baix Vinalopó,Alicante', 'descripcion': '¿Vivir en la cumbre y tener la ciudad a 5 minutos? ¡Es posible! \nDescubre esta exclusiva villa en La Florida, Crevillente, donde la tranquilidad de la naturaleza se combina con la comodidad de la ciudad. Con 575 m² construidos sobre una parcela de 2.580 m², esta vivienda unifamiliar de tres plantas ofrece vistas espectaculares al valle y al paraje natural del Hondo, lejos del ruido urbano. \nCaracterísticas destacadas: \nUbicación privilegiada: A solo 5 minutos de Crevillente, con acceso rápido a servicios, comercios y ocio. \nDiseño rústico y elegante: Integrada en la montaña, con tres plantas (dos sobre rasante y un sótano con acceso para vehículos). \nAmplias terrazas y zonas exteriores: 200 m² de terraza, piscina de 60 m², jacuzzi, bar

In [172]:
# --- MOSTRAR RESULTADOS COMPLETOS ---

print("\n--- DATOS DEL INMUEBLE ---")
print(f"Título: {datos_inmueble['titulo']}")
print(f"Precio: {datos_inmueble['precio']}")
print(f"Ubicación: {datos_inmueble['ubicacion']}")

print("\nDescripción:")
print(datos_inmueble['descripcion'])

print("\nCaracterísticas:")
caracteristicas = datos_inmueble.get('caracteristicas', {})

if caracteristicas:
    for clave, valor in caracteristicas.items():
        print(f"\n· {clave.capitalize()}:")
        if isinstance(valor, list):
            for item in valor:
                print(f"  - {item}")
        elif isinstance(valor, dict):
            for subclave, subvalor in valor.items():
                print(f"  · {subclave.capitalize()}: {subvalor}")
        else:
            print(f"  - {valor}")
else:
    print("No se encontraron características.")




--- DATOS DEL INMUEBLE ---
Título: Casa o chalet independiente en venta en calle San Antonio de la Florida
Precio: 397000
Ubicación: Calle San Antonio de la Florida,Crevillente,Baix Vinalopó,Alicante

Descripción:
¿Vivir en la cumbre y tener la ciudad a 5 minutos? ¡Es posible! 
Descubre esta exclusiva villa en La Florida, Crevillente, donde la tranquilidad de la naturaleza se combina con la comodidad de la ciudad. Con 575 m² construidos sobre una parcela de 2.580 m², esta vivienda unifamiliar de tres plantas ofrece vistas espectaculares al valle y al paraje natural del Hondo, lejos del ruido urbano. 
Características destacadas: 
Ubicación privilegiada: A solo 5 minutos de Crevillente, con acceso rápido a servicios, comercios y ocio. 
Diseño rústico y elegante: Integrada en la montaña, con tres plantas (dos sobre rasante y un sótano con acceso para vehículos). 
Amplias terrazas y zonas exteriores: 200 m² de terraza, piscina de 60 m², jacuzzi, barbacoa y huerta ecológica. 
Interiores es

In [173]:
# 4. Comprobar si la petición fue exitosa (código 200)
if response.status_code == 200:
    print("¡Conexión exitosa! Extrayendo datos...")

    # 5. Parsear el contenido HTML con BeautifulSoup
    # 'soup' es ahora un objeto que podemos "interrogar" para buscar datos.
    soup = BeautifulSoup(response.content, 'lxml')

    # --- EXTRACCIÓN DE DATOS ---

    # a) Extraer el Título
    # Buscamos la etiqueta <h1> que tiene la clase 'main-info__title-main'
    title_element = soup.find_all("h1", class_="main-info__title-main")
    # Limpiamos el texto de espacios extra al principio o al final
    titulo = title_element.text.strip() if title_element else "Título no encontrado"

    # b) Extraer el Precio
    # Buscamos la etiqueta <span> con la clase 'info-data-price'
    price_element = soup.find("span", class_="info-data-price")
    precio = price_element.text.strip() if price_element else "Precio no encontrado"

    # c) Extraer Características Básicas
    # Buscamos el contenedor div con la clase 'details-property-feature-one'
    caracteristicas_container = soup.find("div", class_="details-property-feature-one")

    lista_caracteristicas = []
    if caracteristicas_container:
        # Dentro del contenedor, buscamos TODAS las etiquetas <li> (elementos de la lista)
        items = caracteristicas_container.find_all("li")
        # Recorremos cada item, extraemos su texto y lo añadimos a nuestra lista
        for item in items:
            lista_caracteristicas.append(item.text.strip())

    # --- MOSTRAR RESULTADOS ---

    print("\n--- DATOS DEL INMUEBLE ---")
    print(f"Título: {titulo}")
    print(f"Precio: {precio}")
    print("\nCaracterísticas:")
    if lista_caracteristicas:
        for caracteristica in lista_caracteristicas:
            print(f"- {caracteristica}")
    else:
        print("No se encontraron características básicas.")

else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

¡Conexión exitosa! Extrayendo datos...

--- DATOS DEL INMUEBLE ---
Título: Título no encontrado
Precio: 397.000 €

Características:
- Casa o chalet independiente
- 3 plantas
- 580 m² construidos, 575 m² útiles
- 4 habitaciones
- 4 baños
- Parcela de 2.580 m²
- Terraza y balcón
- Plaza de garaje incluida en el precio
- Segunda mano/buen estado
- Armarios empotrados
- Trastero
- Orientación sur
- Construido en 2001
- Calefacción individual: Bomba de frío/calor
- Acceso y vivienda adaptados para personas con movilidad reducida


In [174]:
# 1. Importar las librerías necesarias
from bs4 import BeautifulSoup
import json  # Para procesar los datos de las fotos
import pprint # Para imprimir el diccionario de forma bonita

# 2. El contenido HTML que proporcionaste
# Asegúrate de pegar todo el código HTML del anuncio dentro de las triples comillas.
html_content = """
<!DOCTYPE html>
<html class="" data-userauth="false" env="es" lang="es" username="">
<!-- PEGA AQUÍ EL HTML COMPLETO -->
</html>
"""

# 3. Parsear el HTML con BeautifulSoup
print("Analizando el código HTML del anuncio...")
soup = BeautifulSoup(html_content, 'lxml')

# 4. Crear un diccionario para almacenar los datos
datos_casa = {}

# --- EXTRACCIÓN DE DATOS ---

# a) PRECIO Y UBICACIÓN
print("Extrayendo precio y ubicación...")
# Usamos .strip() para limpiar espacios en blanco
datos_casa['precio'] = soup.find('span', class_='info-data-price').text.strip()
titulo_main = soup.find('span', class_='main-info__title-main').text.strip()
titulo_minor = soup.find('span', class_='main-info__title-minor').text.strip()
datos_casa['ubicacion'] = f"{titulo_main}, {titulo_minor}"

# b) LINKS DE LAS FOTOS (El método más eficaz)
print("Extrayendo links de las fotos...")
# ¡Aquí está la magia! Los datos de las fotos están en un script, lo que es más fácil de procesar.
scripts = soup.find_all('script')
photo_data_script = None
for script in scripts:
    if script.string and 'adMultimediasInfo' in script.string:
        photo_data_script = script.string
        break

if photo_data_script:
    # Limpiamos el string para que sea un JSON válido
    json_string = photo_data_script.split('adMultimediasInfo = ', 1)[1].strip().rstrip(';')
    multimedia_data = json.loads(json_string)

    # Extraemos las URLs de alta resolución
    fotos = multimedia_data.get('fullScreenGalleryPics', [])
    datos_casa['links_fotos'] = [foto['imageDataService'] for foto in fotos]
else:
    datos_casa['links_fotos'] = ["No se encontraron datos de fotos en el script."]

# c) DATOS ADICIONALES SUGERIDOS
print("Extrayendo datos adicionales sugeridos...")
datos_casa['sugerencias'] = {}

# Descripción del anunciante
descripcion_container = soup.find('div', class_='adCommentsLanguage')
if descripcion_container and descripcion_container.find('p'):
    datos_casa['sugerencias']['descripcion'] = descripcion_container.find('p').get_text(separator='\n').strip()

# Características Básicas
basic_features_div = soup.find('div', class_='details-property-feature-one')
if basic_features_div:
    items = basic_features_div.find_all('li')
    datos_casa['sugerencias']['caracteristicas_basicas'] = [item.text.strip() for item in items]

# Equipamiento
extra_features_div = soup.find('div', class_='details-property-feature-two')
if extra_features_div and extra_features_div.find('h2', string='Equipamiento'):
    ul_equipamiento = extra_features_div.find('h2', string='Equipamiento').find_next_sibling('div').find('ul')
    items_equip = ul_equipamiento.find_all('li')
    datos_casa['sugerencias']['equipamiento'] = [item.text.strip() for item in items_equip]

# Nombre del anunciante y referencia
datos_casa['sugerencias']['anunciante'] = soup.find('a', class_='about-advertiser-name').text.strip()
datos_casa['sugerencias']['referencia'] = soup.find('p', class_='txt-ref').text.strip()


# --- FIN DE LA EXTRACCIÓN ---

# 5. Imprimir el diccionario resultante de forma legible
print("\n--- DATOS EXTRAÍDOS DEL INMUEBLE ---")
pprint.pprint(datos_casa)

Analizando el código HTML del anuncio...
Extrayendo precio y ubicación...


AttributeError: 'NoneType' object has no attribute 'text'